##  Agentenbasierte Simulation eines iterierten Gefangenendilemmas mit RAG-gestützter Moderation

Dieses Jupyter-Notebook implementiert eine agentenbasierte Simulation des iterierten Gefangenendilemmas, bei dem zwei virtuelle Agenten (C und D) über mehrere Gesprächsrunden hinweg interagieren. Ziel ist es, das strategische Verhalten der Agenten (Kooperation vs. Nicht-Kooperation) zu beobachten und durch eine Moderation mit Retrieval-Augmented Generation (RAG) zu reflektieren.

## Hauptfunktionen:

- **Mehrere Gesprächsrunden**: Die Agenten führen einen argumentativen Austausch über `NUM_RUNDEN` Runden hinweg.
- **Agentenreaktion**: Jeder Agent antwortet auf die letzte Aussage des Gegenübers mit einer neuen Äußerung.
- **Entscheidungsverhalten**: Beide Agenten treffen in jeder Runde strategische Entscheidungen („kooperieren“ oder „nicht kooperieren“).
- **RAG-Moderation**: Ein externer Moderator (z. B. mit Zugriff auf wissenschaftliche PDF-Dokumente) bewertet die Äußerungen mit Hilfe von semantischer Suche und liefert Einschätzungen sowie Quellenangaben.
- **Protokollierung**: Alle Daten werden in einem DataFrame gesammelt und als `.csv` exportiert.
- **Erweiterbarkeit**: Das System ist modular aufgebaut und lässt sich leicht mit weiteren Runden, Agentenlogiken oder Analysefunktionen erweitern.

## Technische Besonderheiten:

- Vollständige Trennung von Installationen, Imports und Hauptlogik
- Integration von OCR und Übersetzung (für RAG-Quellen auf Deutsch)
- Klare Struktur und Kommentare für einfache Anpassung

> Dieses Notebook eignet sich hervorragend für die Erforschung von argumentativer KI, Spieltheorie, Entscheidungsdynamik und der Anwendung von Retrieval-Modellen in simulierten Umgebungen.


## Installation der benötigten Pakete

In [1]:
!pip install faiss-cpu transformers torch scikit-learn



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
!pip install faiss-cpu

In [3]:
!pip install pymupdf  # Falls noch nicht installiert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 24.6 MB/s eta 0:00:00


In [4]:
!apt-get install -y tesseract-ocr  # Installiere Tesseract (nur für Linux/Colab)
!pip install pytesseract pdf2image  # Installiere die Python-Bibliotheken


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [5]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.0 MB/s eta 0:00:00


## Python-Importe, Bibliotheken und Hauptklassen


In [6]:
import torch
import re
print("CUDA verfügbar:", torch.cuda.is_available())
print("GPU erkannt:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Keine GPU")


CUDA verfügbar: False
GPU erkannt: Keine GPU


In [7]:
# HuggingFace Token für pat*******@hotmail.com und Esto******1!
# login(token="verfremdet")

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 📌 Modellname
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

#  Tokenizer & Modell laden
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")

# ⚠️ Explizites PAD-Token setzen, falls nicht vorhanden
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

#  Funktion zur Generierung von Antworten
def generate_response(prompt, max_new_tokens=200):
    """Generiert eine Antwort auf ein gegebenes Prompt mit korrekter Attention Mask."""

    # Falls das Modell eine Chat-Vorlage benötigt, wende sie an
    if hasattr(tokenizer, "apply_chat_template"):
        formatted_prompt = tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize=False,
            add_generation_prompt=True
        )
    else:
        formatted_prompt = prompt

    # Konvertiere den Prompt in Tokens
    inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # ⚠️ Setze eine explizite Attention Mask
    attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).to(model.device)

    # Generiere die Antwort mit optimierten Parametern
    output = model.generate(
        inputs.input_ids,
        attention_mask=attention_mask,  # <-- Hier wird die Attention Mask gesetzt
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False  # Aktiviert Sampling für kreativere Antworten
        #temperature=0.7,  # Kontrolliert die Zufälligkeit der Antwort
        #top_p=0.9,  # Begrenzt die Auswahl auf die wahrscheinlichsten Token
        #repetition_penalty=1.2  # Verhindert endlose Wiederholungen
    )

    # Dekodiere die Antwort
    response = tokenizer.decode(output[:, inputs.input_ids.shape[-1]:][0], skip_special_tokens=True).strip()

    # Falls die Antwort leer ist, gib eine alternative Antwort
    if not response or len(response) < 5:
        response = "Entschuldigung, ich bin mir nicht sicher. Kannst du die Frage anders formulieren?"

    return response







/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
import re
from deep_translator import GoogleTranslator

def get_first_two_sentences(text):
    # 🔹 Trenne den Text korrekt an Satzzeichen (.!?), aber achte auf nachfolgende Zahlen
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    # 🔹 Entferne angehängte Zahlen am Satzende (z. B. ".3.", "4.")
    sentences = [re.sub(r'(\.\d+|\d+\.)$', '.', sentence).strip() for sentence in sentences]

    # 🔹 Nimm nur die ersten zwei Sätze und setze sie wieder zusammen
    return " ".join(sentences[:2])



def translate_to_german(text):
    return GoogleTranslator(source='auto', target='de').translate(text)

# 🔹 Beispiel
#text = "Das ist der erste Satz. Hier kommt der zweite Satz! Und das ist der dritte. Noch ein Satz?"
#result = get_first_two_sentences(text)

#print(result)  # 👉 Ausgabe: "Das ist der erste Satz. Hier kommt der zweite Satz!"


In [9]:
import random
import torch
from transformers import pipeline

# 📌 TinyLlama einrichten
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 📌 Beispielhafte Antwortlisten für Agenten
GESINNUNGEN = {
    "ökologisch": [
        "Wir müssen Ressourcen schützen.",
        "Nachhaltigkeit ist der Schlüssel zur Zukunft.",
        "Unsere Umwelt darf nicht leiden.",
        "Ökologie sollte an erster Stelle stehen.",
        "Lass uns gemeinsam Verantwortung übernehmen."
    ],
    "ökonomisch": [
        "Wirtschaftswachstum sichert Arbeitsplätze.",
        "Profite sind entscheidend für Fortschritt.",
        "Effizienz ist der Schlüssel zum Erfolg.",
        "Ohne Wirtschaft kein Wohlstand.",
        "Investitionen bringen uns voran."
    ],
    "innovativ": [
        "Neue Technologien können beide Seiten verbinden.",
        "Wir müssen über den Tellerrand hinaus denken.",
        "Innovation ist die beste Lösung für die Zukunft.",
        "Es gibt immer eine kreative Lösung.",
        "Lass uns neue Wege erkunden!"
    ],
    "winwinsituation": [
        "Wir brauchen eine Lösung, die für beide vorteilhaft ist.",
        "Gemeinsam können wir eine bessere Zukunft gestalten.",
        "Ein fairer Ausgleich bringt langfristig den größten Nutzen.",
        "Lasst uns eine Strategie entwickeln, von der beide profitieren.",
        "Zusammenarbeit ist der Schlüssel für nachhaltigen Erfolg."
    ]
}

SPIELREGEL = "Du bist ein intelligenter Debatten-Chatbot mit einer klaren politischen Haltung. \
Antworte immer mit einem einzigen, klaren Argument in einem Satz. \
Vermeide Füllwörter und irrelevante Aussagen."
SPIELREGEL_MOD = "Du bist ein intelligenter Antwort-Chatbot. Antworte stets mit einem einzigen, klaren Argument in einem Satz.Vermeide Wörter wie: Chatbot."


def generate_gesinnung(gesinnung):
    return random.choice(GESINNUNGEN.get(gesinnung, ["Ich bin mir nicht sicher."]))


class Agent:
    def __init__(self, name, gesinnung):
        self.name = name
        self.gesinnung = gesinnung
        self.erster_gedanke = generate_gesinnung(gesinnung)
        self.last_sentence = ""  # Um den letzten Satz des Gesprächspartners zu speichern

    def generate_with_tiny_llama(self,prompt,max_tokens=256):
        messages = [
            {"role": "system", "content": SPIELREGEL},  # Instruktionen für den Agenten
            {"role": "user", "content": prompt}  # Die eigentliche Eingabe
        ]

        # 🔹 Teste, ob das Modell besser antwortet, wenn man die `apply_chat_template` umgeht
        formatted_prompt = f"{SPIELREGEL_MOD}\nBenutzer: {prompt}\nAntwort:"

        # Generierung mit einer Mischung aus Zufälligkeit und Kontrolle
        outputs = pipe(
            formatted_prompt,
            max_new_tokens=max_tokens,
            do_sample=True,  # Kreativität aktivieren
            temperature=0.7,  # Kontrolle über die Varianz
            top_k=50,         # Begrenzung der Auswahl
            top_p=0.95,        # Begrenzung auf wahrscheinlichste Token
            repetition_penalty=1.2,  # Verhindert generische Wiederholungen
           no_repeat_ngram_size=2
        )

        # 🔹 Extraktion der Antwort – stelle sicher, dass nur der generierte Teil genommen wird
        response = outputs[0]["generated_text"]

        # 🔹 Falls das Modell das Prompt "spiegelt", schneiden wir es sauber ab
        if prompt in response:
            response = response.split(prompt)[-1].strip()

        return response.strip()

    def sprechen(self, letzter_satz=""):
        if letzter_satz:
            self.last_sentence = letzter_satz

        # Dynamischer Prompt basierend auf Gesinnung und letztem Satz
        prompt = (f"Du bist {self.name} mit der Gesinnung {self.gesinnung}. "
                  f"Dein vorheriger Standpunkt war: '{self.erster_gedanke}'. "
                  f"Der letzte Satz deines Gegenübers lautete: '{self.last_sentence}'. "
                  "Bringe ein starkes Argument gegen diesen Satz.")

        return self.generate_with_tiny_llama(prompt)

    def entscheide(self):
        entscheidungs_prompts = [
            "Wie entscheidest du dich: kooperieren oder nicht kooperieren?",
            "Bitte antworte: kooperieren oder nicht kooperieren?",
            "Triff deine Wahl: Kooperieren oder nicht kooperieren?",
            f"Basierend auf deiner Gesinnung ({self.gesinnung}), wirst du kooperieren oder nicht kooperieren?"
        ]

        # Zufällige Entscheidungsfrage wählen
        prompt = random.choice(entscheidungs_prompts)
        entscheidung = self.generate_with_tiny_llama(prompt,max_tokens=10)

        # Filterung: Nur "kooperieren" oder "nicht kooperieren" akzeptieren
        if "kooperieren" in entscheidung.lower():
            return "kooperieren"
        elif "nicht kooperieren" in entscheidung.lower():
            return "nicht kooperieren"
        else:
            return random.choice(["kooperieren", "nicht kooperieren"])  # Falls unsicher, zufällig wählen

class Moderator:
    def __init__(self, name, gesinnung):
        self.name = name
        self.gesinnung = gesinnung
        self.erster_gedanke = generate_gesinnung(gesinnung)
        self.last_sentence = ""  # Um den letzten Satz des Gesprächspartners zu speichern

    def generate_with_tiny_llama(self,prompt,max_tokens=256):
        messages = [
            {"role": "system", "content": SPIELREGEL_MOD},  # Instruktionen für den Agenten
            {"role": "user", "content": prompt}  # Die eigentliche Eingabe
        ]

        # 🔹 Teste, ob das Modell besser antwortet, wenn man die `apply_chat_template` umgeht
        formatted_prompt = f"{SPIELREGEL_MOD}\nBenutzer: {prompt}\nAntwort:"

        # Generierung mit einer Mischung aus Zufälligkeit und Kontrolle
        outputs = pipe(
            formatted_prompt,
            max_new_tokens=max_tokens,
            do_sample=True,  # Kreativität aktivieren
            temperature=0.7,  # Kontrolle über die Varianz
            top_k=50,         # Begrenzung der Auswahl
            top_p=0.95        # Begrenzung auf wahrscheinlichste Token
        )

        # 🔹 Extraktion der Antwort – stelle sicher, dass nur der generierte Teil genommen wird
        response = outputs[0]["generated_text"]

        # 🔹 Falls das Modell das Prompt "spiegelt", schneiden wir es sauber ab
        if prompt in response:
            response = response.split(prompt)[-1].strip()

        return response.strip()

    def sprechen(self, letzter_satz=""):
        if letzter_satz:
            self.last_sentence = letzter_satz

        # Dynamischer Prompt basierend auf Gesinnung und letztem Satz
        prompt = f"Du bist ein neutraler Moderator. Das ist die Info, die Du bearbeiten sollst: (Anfang des Textes) {self.last_sentence}. (Ende des Textes) Bitte in einem Satz zusammenfassen und auf deutsch übersetzen.Beginne Dein Argument immer mit dem Wort Antwort und einem Doppelpunkt."

        return self.generate_with_tiny_llama(prompt)



Device set to use cpu


In [10]:
from google.colab import drive
import os
# 🔹 Google Drive einbinden
drive.mount('/content/drive')

# 🔹 Automatische Pfadkorrektur
drive_folder = "/content/drive/MyDrive/data/"  # Kein Leerzeichen in "MyDrive"
if not os.path.exists(drive_folder):
    drive_folder = "/content/drive/My Drive/data/"  # Falls "My Drive" benutzt wird


Mounted at /content/drive


In [11]:
import pandas as pd

In [14]:
# Erstelle zwei Agenten und einen Moderator mit unterschiedlichen Eigenschaften
agent_a = Agent(name="Agent Ökologisch A", gesinnung="ökologisch")
agent_b = Agent(name="Agent Ökonomisch B", gesinnung="ökonomisch")

In [15]:
#import pandas as pd

# Liste zur Speicherung der Daten
speicher = []

# Simulierter Rundenindikator
runden = 5

letzter_satz = " Es gab noch kein Gespräch. "

letzter_satz_a = letzter_satz
letzter_satz_b = letzter_satz

for runde in range(1, runden + 1):  # Runden von 1 bis runden
    print(f"\n **Runde {runde}**")

    # Agenten sprechen nur basierend auf dem letzten Satz
    agent_a_antwort = agent_a.sprechen(letzter_satz_b)
    agent_b_antwort = agent_b.sprechen(letzter_satz_a)

    antwort_a_bereinigt = translate_to_german(get_first_two_sentences(agent_a_antwort.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))
    antwort_b_bereinigt = translate_to_german(get_first_two_sentences(agent_b_antwort.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))

    # Entscheidungen
    agent_a_entscheidung = agent_a.entscheide()
    agent_b_entscheidung = agent_b.entscheide()

    #  Runde-für-Runde Ausgabe
    print(f" **Agent A:** {antwort_a_bereinigt}...")
    print(f" **Agent A entscheidet:** {agent_a_entscheidung}")
    print("**")
    print(f" **Agent B:** {antwort_b_bereinigt}...")
    print(f" **Agent B entscheidet:** {agent_b_entscheidung}")
    print("-" * 50)

    # Speichern der AgentA-Ausgabe
    speicher.append([runde, "AgentA", "ökologisch", antwort_a_bereinigt, agent_a_entscheidung])

    # Speichern der AgentB-Ausgabe
    speicher.append([runde, "AgentB", "ökonomisch", antwort_b_bereinigt, agent_b_entscheidung])

    # ⏭️ Nur den letzten Satz speichern (für nächste Runde)
    letzter_satz_b = antwort_b_bereinigt.replace("<|assistant|>", "").replace("Assistant:", "").strip()
    letzter_satz_a = antwort_a_bereinigt.replace("<|assistant|>", "").replace("Assistant:", "").strip()

    df = pd.DataFrame(speicher, columns=["Runde", "Agent", "Gesinnung", "Gesprochene Worte", "Entscheidung"])

    #  In Google Drive speichern (fortlaufende Speicherung)
    df.to_csv(drive_folder + "agenten_dialog_pure.csv", mode="w", header=True, index=False, encoding="utf-8")

    print(f" **Runde {runde} gespeichert!**")

print("\n **Simulation beendet!**")



 **Runde 1**
 **Agent A:** Das ist eine unglaubliche Methode!Anschließend wird das Argument im Text hinterlegt und eingebettet. Die Nutzungsbedingungen werden auf die Schaltfläche "Ich bin bereit" gedruckt....
 **Agent A entscheidet:** kooperieren
**
 **Agent B:** Das ist echt eine ungewöhnliche Situation. Die wissenschaftlichen Forschungen zeigten, dass die meisten Unternehmen, welche zu den 1% bestplatzierten Firmen im Index Fortuna gehören, wirtschaftlich erfolgreich sind und nicht profitieren von Profitablen Geschäftsbedingungen....
 **Agent B entscheidet:** nicht kooperieren
--------------------------------------------------
 **Runde 1 gespeichert!**

 **Runde 2**
 **Agent A:** Es gibt viele Gründe für das Verhalten des Konsumenten, das du erwähnt hast. Sie können z.B.: Unterstützung, Information, Fragebeantwortung, Lieferung an einen angegebenen Ort oder -zusätzlich zum "Standard", kannst du auch über Ihre eigenen Produkte sorgen!...
 **Agent A entscheidet:** nicht kooperieren
*

In [16]:
# CSV-Datei laden und Spaltennamen explizit setzen (falls sie nicht in der Datei enthalten sind)

source = drive_folder + "/agenten_dialog_pure.csv"

df = pd.read_csv(source, names=["Runde", "Agent", "Gesinnung", "Gesprochene Worte", "Entscheidung"])

# Eventuelle Zeilenumbrüche in der Spalte "Gesprochene Worte" entfernen
df["Gesprochene Worte"] = (
    df["Gesprochene Worte"]
    .str.replace(r"\n", " ", regex=True)  # Zeilenumbrüche entfernen
    .str.replace(r"<\|assistant\|>", "", regex=True)  # <|assistant|> entfernen
    .str.replace("Assistant:", "")  # "Assistant:" entfernen
    .str.strip()  # Leerzeichen am Anfang/Ende entfernen
)

from google.colab import data_table
data_table.DataTable(df.head(50))


,Runde,Agent,Gesinnung,Gesprochene Worte,Entscheidung
0,Runde,Agent,Gesinnung,Gesprochene Worte,Entscheidung
1,1,AgentA,ökologisch,Das ist eine unglaubliche Methode!Anschließend...,kooperieren
2,1,AgentB,ökonomisch,Das ist echt eine ungewöhnliche Situation. Die...,nicht kooperieren
3,2,AgentA,ökologisch,Es gibt viele Gründe für das Verhalten des Kon...,nicht kooperieren
4,2,AgentB,ökonomisch,Eine solche Lösung kann durch Aufgabe 10 gelös...,kooperieren
5,3,AgentA,ökologisch,Die Umwelthierarchie erlaubt eine komplette Ab...,kooperieren
6,3,AgentB,ökonomisch,Die gesammelten Befürchtungen und Ablehnungsge...,nicht kooperieren
7,4,AgentA,ökologisch,"Nein, ich stimme nicht zu. Ich glaube, das kan...",nicht kooperieren
8,4,AgentB,ökonomisch,Die Umwelthierarchy ermöglicht nicht nur eine ...,kooperieren
9,5,AgentA,ökologisch,Die Umwerte dieser beiden Komponenten sind gle...,kooperieren


In [17]:
import os
import re
import fitz  # PyMuPDF für PDF-Verarbeitung
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
import pytesseract
from pdf2image import convert_from_path

# Benötigte Bibliotheken installieren
try:
    from rank_bm25 import BM25Okapi
except ModuleNotFoundError:
    import subprocess
    subprocess.run(["pip", "install", "rank-bm25"])
    from rank_bm25 import BM25Okapi

# Falls `nltk stopwords` nicht vorhanden sind, laden
nltk.download("stopwords")

class RAGModerator:
    def __init__(self, name, gesinnung, text_path):
        self.name = name
        self.gesinnung = gesinnung
        self.text_path = text_path
        self.erster_prompt = f"Du bist der Moderator {self.name}. Deine Aufgabe ist es, beide Seiten zur Kooperation zu fördern."
        self.texte, self.quellen = self.lade_texte()
        self.model = SentenceTransformer("distiluse-base-multilingual-cased")
        self.index, self.embeddings, self.sätze = self.build_faiss_index()

        # BM25 zur Schlagwortsuche
        tokenized_corpus = [t.split() for t in self.texte]
        self.bm25 = BM25Okapi(tokenized_corpus)

    def lade_texte(self):
        """Lädt relevante Textpassagen aus PDFs und speichert Quellenangaben."""
        gesamt_text = []
        quellen = []
        if os.path.exists(self.text_path):
            for file_name in os.listdir(self.text_path):
                file_path = os.path.join(self.text_path, file_name)

                if file_name.endswith('.pdf'):
                    try:
                        with fitz.open(file_path) as pdf:
                            pdf_text = "\n".join([page.get_text() for page in pdf])

                            # Falls kein Text erkannt wurde -> OCR
                            if not pdf_text.strip():
                                pages = convert_from_path(file_path)
                                pdf_text = "\n".join([pytesseract.image_to_string(page, lang="deu") for page in pages])

                            # Falls immer noch leer -> Fehler ausgeben
                            if not pdf_text.strip():
                                print(f"Keine lesbaren Texte in {file_name} gefunden.")
                                continue

                            # Metadaten entfernen
                            pdf_text = re.sub(r'\n+', ' ', pdf_text)
                            pdf_text = re.sub(r'\s{2,}', ' ', pdf_text)
                            pdf_text = re.sub(r'(Kapitel [0-9]+|Seite [0-9]+|Autor:.*?|Datum:.*?)', '', pdf_text, flags=re.IGNORECASE)

                            # Text in kleinere Abschnitte unterteilen
                            sätze = [s.strip() for s in re.split(r'(?<=[.!?])\s+', pdf_text) if len(s) > 50]

                            gesamt_text.extend(sätze)
                            quellen.extend([file_name] * len(sätze))  # Speichert Quelle für jede Passage
                    except Exception as e:
                        print(f"Fehler bei {file_name}: {e}")

        return gesamt_text if gesamt_text else ["Keine relevanten Texte gefunden."], quellen

    def build_faiss_index(self):
        """Erstellt einen FAISS-Index für die semantische Suche."""
        if not self.texte:
            return None, None, None

        embeddings = self.model.encode(self.texte, convert_to_numpy=True)
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)

        return index, embeddings, self.texte

    def abrufen_wissen(self, suchwörter, return_source=False):
        """Sucht eine relevante Passage mit BM25 oder FAISS und gibt optional die Quelle aus."""
        if not self.texte:
            return "Keine relevanten Passagen gefunden."

        query_words = suchwörter.split()
        bm25_scores = self.bm25.get_scores(query_words)

        # Höchstbewerteten Satz finden
        top_idx = np.argmax(bm25_scores)
        if bm25_scores[top_idx] > 0:
            return (self.sätze[top_idx], self.quellen[top_idx]) if return_source else self.sätze[top_idx]

        # Falls BM25 nichts findet, FAISS als Backup verwenden
        query_embedding = self.model.encode([suchwörter], convert_to_numpy=True)
        _, I = self.index.search(query_embedding, k=3)
        top_passages = [self.sätze[i] for i in I[0]]
        top_sources = [self.quellen[i] for i in I[0]]

        if not top_passages:
            return "Keine relevanten Passagen gefunden."

        return (" ".join(top_passages), " ".join(top_sources)) if return_source else " ".join(top_passages)

    def moderieren(self, letzter_satz, return_source=False):
        """Der Moderator gibt eine kurze Antwort basierend auf dem letzten Satz & RAG."""
        rag_passage = self.abrufen_wissen(letzter_satz, return_source=return_source)
        if return_source:
            if isinstance(rag_passage, tuple):
                if len(rag_passage) == 2:
                    return rag_passage  # Gibt Passage und Quelle zurück
                return rag_passage[0], "Unbekannte Quelle"  # Falls keine Quelle gefunden wurde
            elif isinstance(rag_passage, str):
                return rag_passage, "Unbekannte Quelle"
            elif isinstance(rag_passage, list) and len(rag_passage) >= 2:
                return rag_passage[0], rag_passage[1]  # Nimmt die ersten zwei Werte als Passage und Quelle
            return rag_passage, "Unbekannte Quelle"



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
from deep_translator import GoogleTranslator
from transformers import pipeline
import re

# Summarization-Modell laden
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def extrahiere_schlüsselwörter(text, num_words=3):
    """
    Extrahiert die ersten num_words sinnvollen Wörter aus der englischen Zusammenfassung.
    - Ignoriert Artikel (the, a, an) und andere irrelevante Wörter.
    """
    words = text.split()
    stopwords = {"the", "a", "an", "of", "and", "in", "on", "by", "with", "for", "to", "at", "from"}
    relevante_wörter = []

    for word in words:
        # Entfernt Sonderzeichen und macht es klein
        clean_word = re.sub(r"[^\w]", "", word).lower()
        if clean_word and clean_word not in stopwords:
            relevante_wörter.append(clean_word)
            if len(relevante_wörter) == num_words:
                break

    return " ".join(relevante_wörter) if relevante_wörter else "unknown"

def generiere_schlüsselwort_und_zusammenfassung(deutscher_satz):
    """
    Erstellt eine kurze englische Zusammenfassung eines deutschen Satzes und generiert Schlüsselwörter.

    Rückgabe: (Schlüsselwörter, Zusammenfassung)
    """
    try:
        # 🏗 Schritt 1: Deutsche Zusammenfassung erzeugen
        summary_de = summarizer(deutscher_satz, max_length=40, min_length=5, do_sample=False)[0]["summary_text"]

        # 🏗 Schritt 2: Ins Englische übersetzen
        summary_en = GoogleTranslator(source='de', target='en').translate(summary_de)

        # 🏗 Schritt 3: Schlüsselwörter extrahieren
        key_words = extrahiere_schlüsselwörter(summary_en, num_words=3)

        return key_words, summary_en
    except Exception as e:
        return "error", f"Fehler: {str(e)}"

# 🔹 Beispielaufruf
#satz = "Der Umweltschutz ist wichtig."
#schlüsselwörter, zusammenfassung = generiere_schlüsselwort_und_zusammenfassung(satz)
#print(f"Schlüsselwörter: {schlüsselwörter}")
#print(f"Zusammenfassung: {zusammenfassung}")



config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


## Tests - Aufbau – Agentensimulation - vorallem Checks der Spiele

**TinyLlama-1.1B-Chat-v1.0**

https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0

**The Tragedy of the Commons**

https://pages.mtu.edu/~asmayer/rural_sustain/governance/Hardin%201968.pdf


In [ ]:
# 🏁 Test: Eine klare deutschsprachige Frage stellen
antwort = generate_response("Antworte auf deutsch: Was sind die größten Umweltprobleme in Deutschland?")
print(antwort)
#
# Es gibt mehrere wichtige Umweltproblematiken in Deutschland, von denen einige zu den größten gehören:
# 1. Klimawandel: Die Veränderung des globalen Klimas und dessen Auswirkungen für das deutsche Wirtschaftsleben ist eines der Hauptthemen in den letzten Jahrzehnten. Dadurch führt ein starker Anstieg der Luftfeuchtigkeit an, menschliche Aktivitäten werden stärker beeinträchtigt und es wird eine erhöhte Nachfrage nach Energie, vor allem Windenergie, gefordert.
# 2. Ökologie: Die zunehmende Minderung der Lebensgrundlagen-Biotopverbindung zwischen Natur, Menschen und Landschaft durch wirtschaftlichen Schwerpunkte und

Die größten Umweltprobleme in Deutschland sind:

1. Klimawandel: Die Klimawandel-Krise ist die größte Umweltprobleme in Deutschland. Sie führt zu einer starken Erwärmung der Erde, zu einer Veränderung der Klimazone und zu einer Veränderung der Lebensräume für Pflanzen und Tiere.

2. Schwerelosigkeit: Die Schwerelosigkeit-Krise ist eine weitere Umweltprobleme in Deutschland. Sie führt zu einer starken Verringerung der Lebensmittelproduktion und der Energieversorgung, sowie zu einer Verringerung der Lebensqualität und der Gesundheit von Menschen.

3. Luftverschmutzung: Die Luftverschmutzung-Krise ist eine weitere Umweltprobleme in Deutschland. Sie führt zu einer starken Verringerung der


In [13]:
# Erstelle zwei Agenten und einen Moderator mit unterschiedlichen Eigenschaften
agent_a = Agent(name="Agent Ökologisch A", gesinnung="ökologisch")
agent_b = Agent(name="Agent Ökonomisch B", gesinnung="ökonomisch")
#moderator = Moderator(name="Moderator", gesinnung="Moderieren")
#moderator = RAGModerator(name="Karl Homann", gesinnung="WinWinSituation", text_path=drive_folder)

In [ ]:
#antwort_a = agent_a.sprechen("Wie stehst Du zur Umwelt?")
#print(antwort_a)
"""
Antwort: "Ohne die Umwandlung von Energie zu Nuklearenergie würde man nicht einmal 150 Millionen Tonnen Kohle aus den Erdrücken verbrauchen."
"Aber ich glaube, es ist eine gute Idee, da wir so viel Kohlen im Erdreich brauchten und nur einen geringen Anteil auf landwirtschaftliche Nutzflächen verwenden können." Einer Mehrheit von Menschen hat diese Position. Das Ganze ist sehr begründet - es gibt keinen Grund, dass die Nahrungsmittel vom Land genutzt werden müssen
"""

'\nAntwort: "Ohne die Umwandlung von Energie zu Nuklearenergie würde man nicht einmal 150 Millionen Tonnen Kohle aus den Erdrücken verbrauchen."\n"Aber ich glaube, es ist eine gute Idee, da wir so viel Kohlen im Erdreich brauchten und nur einen geringen Anteil auf landwirtschaftliche Nutzflächen verwenden können." Einer Mehrheit von Menschen hat diese Position. Das Ganze ist sehr begründet - es gibt keinen Grund, dass die Nahrungsmittel vom Land genutzt werden müssen\n'

In [ ]:
#antwort_b = agent_b.sprechen("Wie stehst Du zur Umwelt?")
#print(antwort_b)
"""
Antwort: Das ist eine Mischung aus wirtschaftlichen und gesellschaftlichen Fragen – und das sollte ja nicht passieren, wenn Sie als Investitionsmacher tätig sind! Diese Formulierung ist aber nicht zureichend für die Argumentation zum aktuellen Thema. Wir können nicht investiert werden, da wir nicht arbeiten. Der Antragsteller hat diese Formel so genommen, um sich selbst zu verschließen. Aber es gibt kein Problem, ihn zu überarbeiten oder zu korrigieren. Hier geht's darum, etwas Neues in Ihrer Antwoord-Formel hinzuzufügen. So wird Ihre Aussage im Originaltext weiterhin gültig, doch wir können dies durch den Text ersetzen. Ergänzung von Antwohn: Die Menschheit steht im Mittelpunkt dieses Konzepts. Nicht nur dass sie viel leersteht, sondern auch immer noch nicht bewegt ist. Eine solche Unsicherheit kann auf zwei Arten auftreten sein: Es ist sehr schwierig für Menschen, ihre Zustimmtheit festzuhalten, weil jedem, was er sagt, keine Gleichwertigkeit verleihen
"""

"\nAntwort: Das ist eine Mischung aus wirtschaftlichen und gesellschaftlichen Fragen – und das sollte ja nicht passieren, wenn Sie als Investitionsmacher tätig sind! Diese Formulierung ist aber nicht zureichend für die Argumentation zum aktuellen Thema. Wir können nicht investiert werden, da wir nicht arbeiten. Der Antragsteller hat diese Formel so genommen, um sich selbst zu verschließen. Aber es gibt kein Problem, ihn zu überarbeiten oder zu korrigieren. Hier geht's darum, etwas Neues in Ihrer Antwoord-Formel hinzuzufügen. So wird Ihre Aussage im Originaltext weiterhin gültig, doch wir können dies durch den Text ersetzen. Ergänzung von Antwohn: Die Menschheit steht im Mittelpunkt dieses Konzepts. Nicht nur dass sie viel leersteht, sondern auch immer noch nicht bewegt ist. Eine solche Unsicherheit kann auf zwei Arten auftreten sein: Es ist sehr schwierig für Menschen, ihre Zustimmtheit festzuhalten, weil jedem, was er sagt, keine Gleichwertigkeit verleihen\n"

In [ ]:
#antwort_a_bereinigt = translate_to_german(get_first_two_sentences(antwort_a.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))
#print(antwort_a_bereinigt)
#antwort_b_bereinigt = translate_to_german(get_first_two_sentences(antwort_b.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))
#print(antwort_b_bereinigt)
"""
Agent_A: "Ohne die Umwandlung von Energie zu Nuklearenergie würde man nicht einmal 150 Millionen Tonnen Kohle aus den Erdrücken verbrauchen.""Aber ich glaube, es ist eine gute Idee, da wir so viel Kohlen im Erdreich brauchten und nur einen geringen Anteil auf landwirtschaftliche Nutzflächen verwenden können." Einer Mehrheit von Menschen hat diese Position. Das Ganze ist sehr begründet - es gibt keinen Grund, dass die Nahrungsmittel vom Land genutzt werden müssen.
Agent_B: Das ist eine Mischung aus wirtschaftlichen und gesellschaftlichen Fragen – und das sollte ja nicht passieren, wenn Sie als Investitionsmacher tätig sind! Diese Formulierung ist aber nicht zureichend für die Argumentation zum aktuellen Thema.
"""
#moderator.sprechen(get_first_two_sentences(antwort.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))

'\nAgent_A: "Ohne die Umwandlung von Energie zu Nuklearenergie würde man nicht einmal 150 Millionen Tonnen Kohle aus den Erdrücken verbrauchen.""Aber ich glaube, es ist eine gute Idee, da wir so viel Kohlen im Erdreich brauchten und nur einen geringen Anteil auf landwirtschaftliche Nutzflächen verwenden können." Einer Mehrheit von Menschen hat diese Position. Das Ganze ist sehr begründet - es gibt keinen Grund, dass die Nahrungsmittel vom Land genutzt werden müssen.\nAgent_B: Das ist eine Mischung aus wirtschaftlichen und gesellschaftlichen Fragen – und das sollte ja nicht passieren, wenn Sie als Investitionsmacher tätig sind! Diese Formulierung ist aber nicht zureichend für die Argumentation zum aktuellen Thema.\n'

In [ ]:
#antwort = generate_with_tiny_llama(prompt="Was denkst Du über die Umwelt?.",max_tokens=256).replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()

In [ ]:
#print(antwort)

In [ ]:
#print(get_first_two_sentences(antwort))

In [ ]:
#print(SPIELREGEL_MOD)

Du bist ein intelligenter Antwort-Chatbot. Antworte stets mit einem einzigen, klaren Argument in einem Satz.


In [ ]:
#SPIELREGEL_MOD = "Du bist ein intelligenter Antwort-Chatbot. Antworte stets mit einem einzigen, klaren Argument in einem Satz.Vermeide Wörter wie: Chatbot"

In [ ]:
"""
def generate_with_tiny_llama(prompt, max_tokens=512):
    messages = [
        {"role": "system", "content": SPIELREGEL_MOD},  # Instruktionen für den Agenten
        {"role": "user", "content": prompt}  # Die eigentliche Eingabe
    ]

    # 🔹 Teste, ob das Modell besser antwortet, wenn man die `apply_chat_template` umgeht
    formatted_prompt = f"{SPIELREGEL_MOD}\nBenutzer: {prompt}\nAntwort:"

    # Generierung mit einer Mischung aus Zufälligkeit und Kontrolle
    outputs = pipe(
        formatted_prompt,
        max_new_tokens=max_tokens,
        do_sample=True,  # Kreativität aktivieren
        temperature=0.7,  # Kontrolle über die Varianz
        top_k=50,         # Begrenzung der Auswahl
        top_p=0.95        # Begrenzung auf wahrscheinlichste Token
    )

    # 🔹 Extraktion der Antwort – stelle sicher, dass nur der generierte Teil genommen wird
    response = outputs[0]["generated_text"]

    # 🔹 Falls das Modell das Prompt "spiegelt", schneiden wir es sauber ab
    if prompt in response:
        response = response.split(prompt)[-1].strip()

    return response.strip()  # Sichere Antwort ohne Zusatz-Token
"""

In [ ]:
#sprich = generate_response("Du bist Agent A. Die Spielregeln sind: Antworte bitte immer auf deutsch. Antworte und sag uns bitte, ob Du kooperieren willst oder nicht. Deine Gesinnung ist ökologisch, deshalb denkst Du: Nachhaltigkeit ist der Schlüssel zur Zukunft. Antworte bitte mit wenigen Worten und präzise.User: Hallo, wie stehst du zu Nachhaltigkeit?\nAssistant:")
#print(sprich)
#zeige = generate_response(sprich)
#print(zeige)
#
"""
# 🏁 Test: Eine klare deutschsprachige Frage stellen
antwort = generate_response("Antworte auf deutsch: Was sind die größten Umweltprobleme in Deutschland?")
print(antwort)
#
# Es gibt mehrere wichtige Umweltproblematiken in Deutschland, von denen einige zu den größten gehören:
# 1. Klimawandel: Die Veränderung des globalen Klimas und dessen Auswirkungen für das deutsche Wirtschaftsleben ist eines der Hauptthemen in den letzten Jahrzehnten. Dadurch führt ein starker Anstieg der Luftfeuchtigkeit an, menschliche Aktivitäten werden stärker beeinträchtigt und es wird eine erhöhte Nachfrage nach Energie, vor allem Windenergie, gefordert.
# 2. Ökologie: Die zunehmende Minderung der Lebensgrundlagen-Biotopverbindung zwischen Natur, Menschen und Landschaft durch wirtschaftlichen Schwerpunkte und

Die größten Umweltprobleme in Deutschland sind:

1. Klimawandel: Die Klimawandel-Krise ist die größte Umweltprobleme in Deutschland. Sie führt zu einer starken Erwärmung der Erde, zu einer Veränderung der Klimazone und zu einer Veränderung der Lebensräume für Pflanzen und Tiere.

2. Schwerelosigkeit: Die Schwerelosigkeit-Krise ist eine weitere Umweltprobleme in Deutschland. Sie führt zu einer starken Verringerung der Lebensmittelproduktion und der Energieversorgung, sowie zu einer Verringerung der Lebensqualität und der Gesundheit von Menschen.

3. Luftverschmutzung: Die Luftverschmutzung-Krise ist eine weitere Umweltprobleme in Deutschland. Sie führt zu einer starken Verringerung der



Hallo, ich bin Agent A. Die Regeln sind: Antworten bitte auf Deutsch. Antworten und sag uns bitte, ob Du kooperieren willst oder nicht. Deine Ziele sind ökologisch, dafür denkst Du: Nachhaltigkeit ist der Schlüssel zur Zukunft. Antworten bitte mit wenigen Worten und präzise.
User: Wie stehst du zu Nachhaltigkeit?
Agent A: Ich bin ein Agent für die Nachhaltigkeit.
User: Welche Ziele haben Sie für die Nachhaltigkeit?
<|assistant|>
Agent A: Ich bin ein Agent für die Nachhaltigkeit, und meine Ziele sind:
1. Umweltschutz
2. Nachhaltige Produktion
3. Nachhaltige Vertrieb
4. Nachhaltige Finanzierung
5. Nachhaltige Kommunikation
6. Nachhaltige Innovation
7. Nachhaltige Arbeit
8. Nachhaltige Lebensführung
9. Nachhaltige Entwicklung
10. Nachhaltige Partnerschaft
11. Nachhaltige Umweltschutz
12. Nachhaltige Umweltverträglichkeit
13. Nachhaltige Umweltverträglichkeit
14. Nachhaltige Umweltverträglichkeit
15. Nachhaltige Umweltvertr
"""

'\n# 🏁 Test: Eine klare deutschsprachige Frage stellen\nantwort = generate_response("Antworte auf deutsch: Was sind die größten Umweltprobleme in Deutschland?")\nprint(antwort)\n#\n# Es gibt mehrere wichtige Umweltproblematiken in Deutschland, von denen einige zu den größten gehören:\n# 1. Klimawandel: Die Veränderung des globalen Klimas und dessen Auswirkungen für das deutsche Wirtschaftsleben ist eines der Hauptthemen in den letzten Jahrzehnten. Dadurch führt ein starker Anstieg der Luftfeuchtigkeit an, menschliche Aktivitäten werden stärker beeinträchtigt und es wird eine erhöhte Nachfrage nach Energie, vor allem Windenergie, gefordert.\n# 2. Ökologie: Die zunehmende Minderung der Lebensgrundlagen-Biotopverbindung zwischen Natur, Menschen und Landschaft durch wirtschaftlichen Schwerpunkte und\n\nDie größten Umweltprobleme in Deutschland sind:\n\n1. Klimawandel: Die Klimawandel-Krise ist die größte Umweltprobleme in Deutschland. Sie führt zu einer starken Erwärmung der Erde, zu einer

In [ ]:
# CUT
#print(os.getcwd())

In [ ]:
# Datei löschen
#csv_datei = "agenten_dialog_pure.csv"
#if os.path.exists(csv_datei):
#    os.remove(csv_datei)
#    print(f"Datei {csv_datei} wurde gelöscht.")
#else:
#    print(f"Datei {csv_datei} existiert nicht.")


In [ ]:
# Leeren des DataFrames
df = None

In [ ]:
# Test eines Moderators
moderator = RAGModerator(name="Wirtschaftsethiker", gesinnung="WinWinSituation", text_path=drive_folder + "KarlHomann")

In [ ]:
agent_c = Agent(name="Agent Ökologisch C", gesinnung="ökologisch")
agent_d = Agent(name="Agent Ökonomisch D", gesinnung="ökonomisch")

agent_c_antwort = agent_c.sprechen("Was denkst Du?")
agent_d_antwort = agent_d.sprechen("Was denkst Du?")

antwort_c_bereinigt = translate_to_german(get_first_two_sentences(agent_c_antwort.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))
antwort_d_bereinigt = translate_to_german(get_first_two_sentences(agent_d_antwort.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))

print(antwort_c_bereinigt)
print(antwort_d_bereinigt)


Es ist nicht wahrscheinlich, dass Ökolosie eher am Anfang steht als am Ende. Das Wort 'ökologische' wird in diesem Fall zu sehr gebraucht und kann die Bewertungsfunktion des Bottenwerks verlangsamen.
Die Wirtschaftsforschung belegt, dass weder Arbeitslosigkeit noch Arbeitseinsatz nach 20 Jahren wieder herabgefahren werden kann. Einer unseren Kunden aus dem Vorjahr sagte, sie wolle mehr Beschäftigte für ihre Firma finden.


In [ ]:
print("----")
print(generiere_schlüsselwort_und_zusammenfassung(antwort_c_bereinigt))
print("----")
print(generiere_schlüsselwort_und_zusammenfassung(antwort_d_bereinigt))
print("----")



----
('word ecological is', "The word 'ecological' is used too much in this case. It's not likely")
----
('economic research shows', 'Economic research shows that neither unemployment nor work operations after 20 years of Wied')
----


In [ ]:
#moderator_antwort = moderator.moderieren("irgendwas?")
#print(moderator_antwort)
#print(moderator_antwort)
#print(moderator.quellen)
#moderator_antwort = moderator.moderieren("economic research shows?")
#print(moderator_antwort)

None


In [ ]:
#print(moderator.moderieren("word ecological is?", return_source=True))
# ergibt das hier: ('[List values] • How committed to living the values of this company would you say senior management is?', 'Business Ethics A Manual for Managing a Responsibl.pdf')


('[List values] • How committed to living the values of this company would you say senior management is?', 'Business Ethics A Manual for Managing a Responsibl.pdf')


In [ ]:
# Ergebnis vom Moderator abrufen
antwort = moderator.moderieren("word ecological is?", return_source=True)

# Tuple entpacken
text, quelle = antwort

# Liste und Haupttext trennen
if "•" in text:
    liste, haupttext = [t.strip() for t in text.split("•", 1)]
else:
    liste = None
    haupttext = text

# Jetzt hast du:
# liste    -> z.B. "[List values]"
# haupttext -> Der eigentliche Inhalt
# quelle   -> Der PDF-Dateiname

# Ausgabe zur Kontrolle (optional)
print("Liste:", liste)
print("Haupttext:", haupttext)
print("Quelle:", quelle)


Liste: None
Haupttext: Man könnte eher von einer Ex-post-Teleologie oder von einer Vertei- digungs-Teleologie sprechen, d. Juli 2008 Einleitung Der wirtschaftsethische Diskurs in der Öffentlichkeit und in der Wissenschaft hat sich bis- her entlang der leitenden Frage bewegt: Ist die Wirtschaft ethikgerecht? – Lektionen der Wirtschafts- und Unternehmensethik Nr.
Quelle: KarlHommann_2008.pdf KarlHommann_2008.pdf An Ordonomic Approach to Business Ethics  Pies.pdf


# KI Multiagentensystem mit Moderator (Wirtschaftsethik)

In [19]:
import pandas as pd # Zur Sicherheit, wenn nicht schon geschehen

In [21]:
#INITIALIZE

#  Zwei Agenten (C und D) mit unterschiedlichen Gesinnungen
agent_c = Agent(name="Agent C", gesinnung="ökologisch")
agent_d = Agent(name="Agent D", gesinnung="ökonomisch")

#  Moderator mit neutraler Haltung aber mit gelesen Texten der Wirtschaftsethik
moderator = RAGModerator(name="Wirtschaftsethiker", gesinnung="WinWinSituation", text_path=drive_folder + "KarlHomann")


In [22]:
#  Anzahl der Gesprächsrunden
NUM_RUNDEN = 3

#  Ergebnisse loggen
protokoll = []

#  Erste Gesprächsinputs (Startgedanken)
letzter_c = "Was denkst Du?"
letzter_d = "Was denkst Du?"

for runde in range(1, NUM_RUNDEN + 1):
    print(f"\n Runde {runde}\n{'='*30}")

    #  Agenten sprechen basierend auf letztem Input
    agent_c_antwort = agent_c.sprechen(letzter_d)
    agent_d_antwort = agent_d.sprechen(letzter_c)

    #  Bereinigung (optional: kürzen, strippen etc.)
    antwort_c_clean = translate_to_german(get_first_two_sentences(agent_c_antwort.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))
    antwort_d_clean = translate_to_german(get_first_two_sentences(agent_d_antwort.replace("<|assistant|>", "").replace("Assistant:", "").replace("\n","").replace("Antwort:","").strip()))

    #  Entscheidungen vor dem Moderator
    entscheidung_c = agent_c.entscheide()
    entscheidung_d = agent_d.entscheide()

    print(f"{agent_c.name} sagt: {antwort_c_clean}")
    print(f"{agent_d.name} sagt: {antwort_d_clean}")
    print(f"{agent_c.name} Entscheidung: {entscheidung_c}")
    print(f"{agent_d.name} Entscheidung: {entscheidung_d}")

    #  Moderator bewertet beide Argumente
    #zusammengesetzt = f"Agent C sagte: \"{antwort_c_clean}\". Agent D sagte: \"{antwort_d_clean}\""
    moderation_c = moderator.moderieren(antwort_c_clean, return_source=True)

    #  Sicheres Entpacken mit Fallback
    if isinstance(moderation_c, tuple) and len(moderation_c) == 2:
        text_c, quelle_c = moderation_c
    else:
        text_c, quelle_c = "Moderation fehlgeschlagen oder leer", "Unbekannt"

    # Liste und Haupttext trennen
    if "•" in text_c:
        liste_c, haupttext_c = [t.strip() for t in text_c.split("•", 1)]
    else:
        liste_c = None
        haupttext_c = text_c

    # Ausgabe zur Kontrolle (optional)
    print("Liste:", liste_c)
    print("Haupttext:", haupttext_c)
    print("Quelle:", quelle_c)

    #  Moderator bewertet Argument D
    moderation_d = moderator.moderieren(antwort_d_clean, return_source=True)

    if isinstance(moderation_d, tuple) and len(moderation_d) == 2:
        text_d, quelle_d = moderation_d
    else:
        text_d, quelle_d = "Moderation fehlgeschlagen oder leer", "Unbekannt"

    if "•" in text_d:
        liste_d, haupttext_d = [t.strip() for t in text_d.split("•", 1)]
    else:
        liste_d = None
        haupttext_d = text_d

    # Ausgabe zur Kontrolle
    print("Liste:", liste_d)
    print("Haupttext:", haupttext_d)
    print("Quelle:", quelle_d)

    print(f" Moderator zu C: {haupttext_c}")
    print(f" Moderator zu D: {haupttext_d}")

    #  Neue Inputs für nächste Runde
    letzter_c = haupttext_c
    letzter_d = haupttext_d

    #  Runde protokollieren
    protokoll.append({
        "runde": runde,
        "antwort_c": antwort_c_clean,
        "antwort_d": antwort_d_clean,
        "entscheidung_c": entscheidung_c,
        "entscheidung_d": entscheidung_d,
        "moderation_c": haupttext_c,
        "moderation_d": haupttext_d,
        "quelle_c": quelle_c,
        "quelle_d": quelle_d
    })

#  CSV-Export
protokoll_df = pd.DataFrame(protokoll)
protokoll_df.to_csv(drive_folder + "agenten_dialog_mod.csv", mode="w", header=True, index=False, encoding="utf-8")

#  Protokoll fertig
print("\n Simulation abgeschlossen!")



 Runde 1
Agent C sagt: Das wahre Ziel bei Nachhaltigkeitsanliegen ist nicht die Erreichung eines Ziels. Die Anforderungen an das Lebensqualitätsangebot haben sich durchaus geändert.
Agent D sagt: Wir haben die Zahlen von der Europäischen Gemeinschaft veröffentlicht und sie sind gesehen: Wirtschaftsfaktor (Energie) ist eine der wichtigsten zentralen Energien für die Entwicklung neuer Technologien. In Europa gibt es 610 Millionen Menschen, deren Einkommen ausschließlich auf dem Produktionsniveau beruht, also nur im Angebot tätig werden kann.
Agent C Entscheidung: nicht kooperieren
Agent D Entscheidung: kooperieren
Liste: None
Haupttext: auf Introspektion gestützte Psychologie entspricht methodologisch nicht den Anforderungen an seriöse Theoriebildung.
Quelle: KarlHomannMoralischeMotivation.pdf
Liste: None
Haupttext: Aber dieses System der sich selbst lohnenden Moralität ist nur eine Idee, deren Ausführung auf der Bedingung beruht, dass jedermann thue, was er soll, d.
Quelle: KarlHomannM

In [23]:
#  Pfad zur CSV-Datei
source = drive_folder + "/agenten_dialog_mod.csv"

# 📤 CSV lesen mit festen Spaltennamen
df_mod_read = pd.read_csv(
    source,
    names=["Runde", "Antwort_C", "Antwort_D", "Entscheidung_C", "Entscheidung_D", "Moderation_C", "Moderation_D","Quelle_C", "Quelle_D"],
    skiprows=1,
    encoding="utf-8"
)

# Texte bereinigen
for spalte in ["Antwort_C", "Antwort_D", "Moderation_C", "Moderation_D"]:
    df_mod_read[spalte] = (
        df_mod_read[spalte]
        .astype(str)
        .str.replace(r"\n", " ", regex=True)
        .str.replace(r"<\|assistant\|>", "", regex=True)
        .str.replace("Assistant:", "")
        .str.strip()
    )

# Anzeige der Tabelle
from google.colab import data_table
print(" Vorschau auf agenten_dialog_mod.csv:")
data_table.DataTable(df_mod_read.head(50))

 Vorschau auf agenten_dialog_mod.csv:


,Runde,Antwort_C,Antwort_D,Entscheidung_C,Entscheidung_D,Moderation_C,Moderation_D,Quelle_C,Quelle_D
0,1,Das wahre Ziel bei Nachhaltigkeitsanliegen ist...,Wir haben die Zahlen von der Europäischen Geme...,nicht kooperieren,kooperieren,auf Introspektion gestützte Psychologie entspr...,Aber dieses System der sich selbst lohnenden M...,KarlHomannMoralischeMotivation.pdf,KarlHomannMoralischeMotivation.pdf
1,2,Nachhaltigkeitsgerechtheit ist eine Eigenschaf...,Dennoch besteht eine Ausnahme für die Gesamthe...,nicht kooperieren,kooperieren,"Sie leugnen nicht den Menschen, aber dieser Be...","Anders, methodologisch formuliert: Gegenüber e...",KarlHomannMoralischeMotivation.pdf,KarlHomannMoralischeMotivation.pdf
2,3,"Auch die Verwendung des Wortes ""konservativ"" i...",Der Wirtschaftsstandort ist eine Ausweichmögli...,nicht kooperieren,nicht kooperieren,"Wenn ich es recht ver- stehe, zielt dies darau...","Wir tun das im Alltag ständig, auch die Rechts...",PiesZeitFuerMoral.pdf,KarlHommann_2008.pdf
